In [48]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math



In [49]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [50]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products.pkl')
interaction_data = interaction_data.sample(1000)
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
interaction_data.main_product_id = interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(interaction_data, product_data)

,customer_id,main_product_id,amount
550974,10666965,7701616,1
469961,9609324,3476869,1
474908,9666261,4075106,1
113723,4110264,3711758,1
280172,6636578,549776,1
...,...,...,...
460948,9506506,3711727,1
515092,10131421,4076652,1
346192,7720219,3711727,1
48178,2090772,4913521,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [51]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': interaction_data['customer_id'].values,
    'main_product_id': interaction_data['main_product_id'].values,
    'amount': interaction_data['amount'].values
})
#variables 
dataset_len = interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 16
train_batch_size = 128
test_batch_size = 64
random_seed = 27

In [52]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [53]:
# Define a model using TensorFlow Recommenders
class RetailModel(tfrs.Model):

    def __init__(self, unique_product_ids, unique_customer_ids, embedding_dimension=32):
        super().__init__()
        # Set up user and product representations
        self.user_embedding = Embedding(unique_customer_ids.shape[0] + 1, embedding_dimension)
        self.product_embedding = Embedding(unique_product_ids.shape[0] + 1, embedding_dimension)
        # Set up a dense layer for the task.
        self.dense_layer = Dense(128, activation="relu")
        product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)

        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model)
                                                )
          
        )
        self.auc_metric = AUC(name='auc')

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """
        Computes the dot product between user and product embeddings to get the interaction score.
        """
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # print(features)
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])
        
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)
        # This is a hypothetical function that returns the logits or scores
        # This needs to be adapted based on your actual model
        positive_logits = self.dot_product_score(user_output, product_output)

        # Update AUC
        self.auc_metric.update_state(y_true=features["amount"], y_pred=positive_logits)  # assuming you have a binary "label" in your dataset indicating 1 for positive interaction and 0 for negative

        return self.task(user_output, product_output)

In [54]:
# Unique customer and product identifiers
unique_customer_ids = interaction_data["customer_id"].unique()
unique_product_ids = interaction_data["main_product_id"].unique()
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [55]:
model = RetailModel(unique_product_ids, unique_customer_ids, embedding_dimension=128)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001)) # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [56]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=3)

Epoch 1/3
7/7 [==============================] - 2s 215ms/step - auc: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 493.5208 - regularization_loss: 0.0000e+00 - total_loss: 493.5208
Epoch 2/3
7/7 [==============================] - 1s 192ms/step - auc: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 493.5208 - regularization_loss: 0.0000e+00 - total_loss: 493.5208
Epoch 3/3
7/7 [==============================] - 1s 191ms/step - auc: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - facto

In [57]:
# Evaluate the model
result_evaluate = model.evaluate(cached_train)
result_evaluate

7/7 [==============================] - 2s 222ms/step - auc: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 1.0000 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 493.5208 - regularization_loss: 0.0000e+00 - total_loss: 493.5208


[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 110.90354919433594, 0, 110.90354919433594]

In [58]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
auc		0.0
factorized_top_k/top_1_categorical_accuracy		1.0
factorized_top_k/top_5_categorical_accuracy		1.0
factorized_top_k/top_10_categorical_accuracy		1.0
factorized_top_k/top_50_categorical_accuracy		1.0
factorized_top_k/top_100_categorical_accuracy		1.0
AUC: 0.0
